In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
offers = pd.read_csv("../data/laraData/offer_events.csv", sep = ';')
applications = pd.read_csv("../data/laraData/application_events.csv", sep = ';')
workitems = pd.read_csv("../data/laraData/workitem_events.csv", sep = ';')
events = pd.read_csv("../data/laraData/events.csv", sep = ';', low_memory = False)

In [3]:
workitems.head()

,A_ID,WI_HIST_ID,WI_HIST_DT_CREATED,WI_ID,WI_USER_CREATED,WI_ACTION,WI_ACTION_DESC,WI_QUEUE,WI_QUEUE_DESC,event,...,NUMBER_OF_CHILDREN,MARITAL_STATUS,SEX,HOUSING_TYPE,INCOMEAMOUNT_YEAR,SCE_CREDITSCORE,SCE_ACCEPTED,SCE_ACCEPTED_OVERRIDE,SCE_MAX_AMOUNT,SCE_MAX_AMOUNT_OVERRIDE
0,719696,2756085,"4/1/2017 19:18:29,800000",512175,USER_31,944,Verkregen,25,Nabellen incomplete dossiers,W_Call incomplete files,...,0.0,Samenwonend,Vrouw,Koophuis,41088.0,784.0,1.0,1.0,49340.0,49340.0
1,719696,2754294,"4/1/2017 13:07:45,692000",512175,USER_100,944,Verkregen,25,Nabellen incomplete dossiers,W_Call incomplete files,...,0.0,Samenwonend,Vrouw,Koophuis,41088.0,784.0,1.0,1.0,49340.0,49340.0
2,719696,2757702,"5/1/2017 11:15:24,925000",512842,USER_119,942,Aangemaakt,24,Valideren aanvraag,W_Validate application,...,0.0,Samenwonend,Vrouw,Koophuis,41088.0,784.0,1.0,1.0,49340.0,49340.0
3,719696,2761252,"6/1/2017 10:17:19,545000",512842,USER_100,943,Verwijderd,24,Valideren aanvraag,W_Validate application,...,0.0,Samenwonend,Vrouw,Koophuis,41088.0,784.0,1.0,1.0,49340.0,49340.0
4,719696,2757704,"5/1/2017 11:15:50,183000",512842,USER_119,945,Vrijgeven,24,Valideren aanvraag,W_Validate application,...,0.0,Samenwonend,Vrouw,Koophuis,41088.0,784.0,1.0,1.0,49340.0,49340.0


In [4]:
applications.head()

,A_ID,A_DT_CREATED,A_INIT_REQ_AMT,A_USER_CREATED,A_APPLICATIONTYPE,A_APPLICATIONTYPE_DESC,A_LOANGOALTYPE,A_LOANGOALTYPE_DESC,AS_ID,AS_APPLICATIONSTATE,...,NUMBER_OF_CHILDREN,MARITAL_STATUS,SEX,HOUSING_TYPE,SCE_CREDITSCORE,SCE_ACCEPTED,SCE_ACCEPTED_OVERRIDE,SCE_MAX_AMOUNT,SCE_MAX_AMOUNT_OVERRIDE,INCOMEAMOUNT_YEAR
0,719696,"1/1/2017 2:42:29,064000",29000.0,USER_1,520,Nieuw krediet,866.0,Auto,1453642,511,...,0.0,Samenwonend,Vrouw,Koophuis,784.0,1.0,1.0,49340.0,49340.0,41088.0
1,719696,"1/1/2017 2:42:29,064000",29000.0,USER_1,520,Nieuw krediet,866.0,Auto,1454067,513,...,0.0,Samenwonend,Vrouw,Koophuis,784.0,1.0,1.0,49340.0,49340.0,41088.0
2,719696,"1/1/2017 2:42:29,064000",29000.0,USER_1,520,Nieuw krediet,866.0,Auto,1458788,517,...,0.0,Samenwonend,Vrouw,Koophuis,784.0,1.0,1.0,49340.0,49340.0,41088.0
3,719696,"1/1/2017 2:42:29,064000",29000.0,USER_1,520,Nieuw krediet,866.0,Auto,1456650,516,...,0.0,Samenwonend,Vrouw,Koophuis,784.0,1.0,1.0,49340.0,49340.0,41088.0
4,719696,"1/1/2017 2:42:29,064000",29000.0,USER_1,520,Nieuw krediet,866.0,Auto,1453643,512,...,0.0,Samenwonend,Vrouw,Koophuis,784.0,1.0,1.0,49340.0,49340.0,41088.0


In [5]:
created = applications.loc[applications.event == "A_Create Application"]
print(created.shape[0])
print(len(list(set(created.A_ID))))

submitted = applications.loc[applications.event == "A_Submitted"]
print(submitted.shape[0])
print(len(list(set(submitted.A_ID))))

concept = applications.loc[applications.event == "A_Concept"]
print(concept.shape[0])
print(len(list(set(concept.A_ID))))

25467
25467
54709
54709
45679
45678


A_Concept denotes the first automatic check that is done online. Let's have a look at the applications that fail this particular test.

<font color='red'><i>Critical step: Since we are dealing with timestamped data, it can happen that there applications whose A_Concept event is in the given dataset, while its A_Submitted event is not in the given dataset</i></font>

In [6]:
applications_submitted = list(submitted.A_ID)
applications_concept = list(set(applications_submitted).intersection(set(concept.A_ID)))
print(len(applications_concept))
applications_fail_test = list(set(applications_submitted).difference(set(applications_concept)))
len(applications_fail_test)

34095


20614

In [7]:
came_through = list(set(applications.loc[applications.event == 'A_Pending'].A_ID))
len(set(applications_fail_test).intersection(set(came_through)))# just to double-check

0

In [8]:
fail = applications.loc[applications.A_ID.isin(applications_fail_test)]
fail = fail.drop_duplicates(subset='A_ID', keep="last")
fail.shape[0]

20614

In [9]:
success = applications.loc[~applications.A_ID.isin(applications_fail_test)]
success = success.drop_duplicates(subset='A_ID', keep="last")
success.shape[0]

45678

In [10]:
accept = applications.loc[applications.event == 'A_Pending']
accept.shape, len(set(accept.A_ID)) # obviously no need to drop duplicates

((13571, 27), 13571)

<font color='red'><i>Since we want to compare the applications/applicants who fail the initial tests, we have to remove these applications from the 'reject-controlgroup'. This is due to the fact that the applications that are immediately rejected, also get an event record with the 'A_Denied'/'A_Cancelled' label.</i></font>

In [11]:
reject = applications.loc[applications.event.isin(['A_Denied', 'A_Cancelled']) & ~applications.A_ID.isin(applications_fail_test)]
reject.shape

(31733, 27)